<a href="https://colab.research.google.com/github/luanc202/data-mining-tb-chest-analysis/blob/main/x_ray_chest_data_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !apt-get install -y git

!git clone https://github.com/luanc202/data-mining-tb-chest-analysis

Cloning into 'data-mining-tb-chest-analysis'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 5), reused 22 (delta 5), pack-reused 0
Receiving objects: 100% (22/22), 115.52 KiB | 2.82 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# bibliotecas
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from traitlets import validate
import sklearn.model_selection
import numpy as np
import tensorflow as tf
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparo do path para os dados

Vamos definir o caminho para as imagens e carregar os caminhos em variáveis.
Após feita a leitura dos caminhos e salvo numa lista de string onde cada item é o caminho para uma imagem, vamos alocar para dataframes, um para cada classe.

Não entendi o porquê de atribuir um label com um número inteiro arbitrário, mas optei por manter por estar presente na maioria dos estudos que encontrei. Provavelmente é para diferenciar as classes.

Ao final, cria-se um dataframe com ambas classes e fazemos a leitura da imagem como matriz pela função *lambda* passada para o apply, redimensionamento da matriz imagem para 224x224 e inserida a matriz imagem redimesionada no dataframe na coluna de `mat_images`.

In [ ]:
# definindo o caminho das imagens
DATA_PATH="TB_Chest_Radiography_Database"
DATA_TRAIN_NORNAL=DATA_PATH+"/Normal/"
DATA_TRAIN_Tuberculosis=DATA_PATH+"/Tuberculosis/"

Normal_img=os.listdir(DATA_TRAIN_NORNAL)
Tuberculosis_img=os.listdir(DATA_TRAIN_Tuberculosis)
# DATA2_TRAIN_NORNAL=DATA2_TRAIN_NORNAL+Normal_img
# DATA2_TEST_Tuberculosis=DATA2_TEST_Tuberculosis+Tuberculosis_img
dnorm=pd.DataFrame(Normal_img,columns=["images"])
dnorm["labels"]=0
dnorm["images"]=DATA_TRAIN_NORNAL+dnorm["images"]

dtuber=pd.DataFrame(Tuberculosis_img,columns=["images"])
dtuber["labels"]=1
dtuber["images"]=DATA_TRAIN_Tuberculosis+dtuber["images"]


DF2=pd.concat([dnorm,dtuber],ignore_index=True)
DF2["mat_images"]=DF2['images'].apply(lambda x: cv2.resize(cv2.imread(x,0) ,( 224, 224)))
DF2.sample(frac=1, random_state=42).reset_index(drop=True)

## Informações sobre o Dataframe

In [ ]:
DF2.info()

## Descrição

In [ ]:
DF2.describe()

## Distribuição por classe

In [ ]:
cases_count = DF2['labels'].value_counts()
print(cases_count)

plt.figure(figsize=(10,8))
sns.barplot(x=cases_count.index, y= cases_count.values)
plt.title('Distribuição', fontsize=14)
plt.xlabel('Classe', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.xticks(range(len(cases_count.index)), ['Normal(0)', 'Tuberculosis(1)'])
plt.show()